In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

## 10 categorías que tienen la menor cantidad de contenido anónimo publicado.

Cargo los CSV de contents y de categoryLinks como DF y luego los paso a RDD

In [4]:
df_contents=pd.read_csv('contents.csv', encoding='Latin-1')
df_category=pd.read_csv('categoryLinks.csv', encoding='Latin-1')

In [5]:
df_contents = df_contents.sample(frac=0.5)
df_contents.to_parquet('contents_p16.parquet')
df_category.to_parquet('categoryLinks.parquet')

Reinicio el Kernel y levanto los archivos .parquet para liberar memoria y poder correr el ultimo Join y takeOrdered

In [3]:
df_contents=sqlContext.read.parquet('contents_p16.parquet')
df_category=sqlContext.read.parquet('categoryLinks.parquet')

In [4]:
rdd_contents=df_contents.rdd.cache()
rdd_category=df_category.rdd.cache()

Hago un map del RDD de category dejando la estructura id como clave y nombre de la categoria como valor

A su vez, hago un map en el RDD de contents para que me quede la estructura id como clave y las IP como valor ya que me piden todo el contenido anonimo publicado y esto lo hacen usuarios no registrados, por lo tanto que aparece su IP.
Una vez hecho esto, hago un filter por el valor(IP) que no son None asi descarto los ID que pertenecen a contenido publicado por usuarios registrados

Para terminar eealizo el Join por ID de las categorias con los contenidos, quedandome la forma id como clave y titulo de la categoria y las ip como valores. Luego realizo un nuevo map moviendo el titulo como clave y un 1 en el valor para poder hacer el ReduceBYKey para que me sumen las repeticiones de cada categoria.

In [5]:
rdd_category_map = rdd_category.map(lambda x: (x[0],x[1]))
rdd_contents_map = rdd_contents.map(lambda x: (x[1],x[8])).filter(lambda x: x[1] != None)
top10_categories = rdd_category_map.join(rdd_contents_map).map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y)


Realizo el takeOrdered 10 para que me traiga los primeros 10 registros que van a ser los de menor cantidad de publicaciones anonimas

In [6]:
top10_categories.takeOrdered(10, lambda x:x[1])

[('Militares_de_Aguascalientes', 1),
 ('Obras_de_William_Shakespeare', 1),
 ('Localidades_del_condado_de_La_Plata', 1),
 ('Arquitectos_de_la_ciudad_de_Buenos_Aires', 1),
 ('Urbanistas_de_Argentina', 1),
 ('Organizaciones_religiosas_de_Arabia_Saudita', 1),
 ('Tipos_de_oraciÃ\x83Â³n', 1),
 ('Historia_de_la_agricultura_en_Venezuela', 1),
 ('Venezuela_en_1834', 1),
 ('Obras_de_1966', 1)]